<img src="https://opea.dev/wp-content/uploads/sites/9/2024/04/opea-horizontal-color.svg" alt="OPEA Logo">

<div style="text-align: center; margin: 20px 0;">
<a href="https://colab.research.google.com/github/opea-project/Course-Material/blob/main/Curriculas/Kubernetes/AgentQnA/3_Add_a_new_agent.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
</div>

> **Quick Start:** Click the button above to open this notebook directly in Google Colab - no local setup required!

# Add Your Own Web Search Agent

## Extending Agent Capabilities with Real-Time Web Search

---

### Learning Objectives

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 15px; border-radius: 10px; color: white; margin: 10px 0;">
<strong>By the end of this tutorial, you'll be able to:</strong>
<ul>
<li><strong>Design</strong> - Create new agent interfaces and tool definitions</li>
<li><strong>Implement</strong> - Build web search functionality with Google APIs</li>
<li><strong>Integrate</strong> - Connect new agents with the supervisor system</li>
<li><strong>Deploy</strong> - Add new microservices to your Kubernetes cluster</li>
<li><strong>Test</strong> - Validate real-time information retrieval capabilities</li>
</ul>
</div>

# Prerequisites

This notebook assumes you have successfully completed **Notebook 1: Deploy AgentQnA** and have:

## 1. **Running AgentQnA Deployment**
   - All AgentQnA pods are running in your Kubernetes cluster
   - Supervisor, RAG Agent, and SQL Agent are operational
   - Port forwarding configured for accessing services

## 2. **Required Tools Available**
   - **`kubectl`** - Configured and connected to your cluster
   - **`curl`** and **`jq`** - For testing API endpoints
   - **`docker`** - If using KIND cluster for file mounting

## 3. **Agent Configuration Files**
   - Agent tools and configuration files should be accessible
   - `/mnt/tools/` directory mounted in your cluster (for KIND)

## 4. **External API Access**
   - **Google Cloud Account** - For Search API access
   - **Internet connectivity** - For API calls and web search functionality

## 5. **Development Skills**
   - Basic understanding of **Python** and **YAML**
   - Familiarity with **REST APIs** and **JSON**
   - Experience with **Kubernetes deployments**

---

# 1. Introduction: Why Add a Web Search Agent?

## 1.1 From Static to Dynamic: The Need for Real-Time Information

In our previous notebooks, we worked with static knowledge bases that contain pre-ingested information. While this approach works well for many use cases, **real-world applications often require access to current, up-to-date information** that changes frequently.

Examples of time-sensitive information needs:
- **Concert Schedules**: Tour dates are announced, venues change, shows are cancelled
- **News Events**: Breaking news, current events, and trending topics
- **Market Data**: Stock prices, exchange rates, and financial indicators
- **Weather Information**: Current conditions, forecasts, and alerts

### Key Limitations of Static Knowledge:

| Limitation | Problem | Impact |
|------------|---------|---------|
| **Outdated Information** | Training data becomes stale over time | Inaccurate responses to current events |
| **No External Access** | Isolated from real-time data sources | Limited to pre-existing knowledge |
| **Static Datasets** | Cannot adapt to changing information | Responses may be completely wrong |
| **Manual Updates** | Requires reingestion for new data | Slow adaptation to new information |

---

## 1.2 Solution: Web Search Agent

To overcome these limitations, we'll create a **Web Search Agent** that can:

1. **Access Real-Time Information** - Query current data from the internet
2. **Provide Accurate Answers** - Get up-to-date information about events, schedules, and news
3. **Seamless Integration** - Work alongside existing agents in the supervisor system
4. **Fast Response** - Retrieve information quickly through APIs

### Our Use Case: Concert Information

We'll focus on **concert and event information** because it demonstrates the need for real-time data:
- Tour announcements happen frequently
- Venue changes and cancellations are common
- Ticket availability changes in real-time
- Artist schedules are constantly updated

---




# 2. Setup and Environment Verification

Let's start by verifying that your environment is properly set up and that the Agent Supervisor is accessible.

## 2.1 Port-Forward Verification Process

The code below will:

1. **Check Connection**: Verify if port 9090 is already accessible
2. **Start Port-Forward**: Automatically start port-forwarding if needed
3. **Validate Access**: Confirm the Agent Supervisor is reachable
4. **Error Handling**: Provide clear feedback on connection status

We need a stable connection to the Agent Supervisor to test our current system limitations and validate our new web search agent later.

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 15px; border-radius: 10px; color: white; margin: 10px 0;">
<strong>Important:</strong> This step ensures we have a baseline working system before adding new capabilities. If this fails, please review your Kubernetes cluster setup and ensure AgentQnA is properly deployed.
</div>


In [ ]:
#Check if your Agent supervisor has the port-forwarding enabled 

import socket 

import socket
import subprocess
import time

# Check if port 9090 is already open
def is_port_open(host='localhost', port=9090):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.settimeout(1)  # short timeout for responsiveness in notebook
        try:
            sock.connect((host, port))
            return True
        except:
            return False

if is_port_open():
    print("✅ Port-forward already running on localhost:9090.")
else:
    print("🔁 Port not open. Starting port-forward to svc/agentqna-supervisor...")
    subprocess.Popen(
        ["kubectl", "port-forward", "svc/agentqna-supervisor", "9090:9090"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    # Optional: wait a moment to let it start
    time.sleep(2)
    if is_port_open():
        print("✅ Port-forward started successfully.")
    else:
        print("❌ Failed to start port-forward.")


# 3. Understanding Current System Limitations



## 3.1 Testing Static Knowledge Boundaries

Let's start by understanding why we need a new agent. We'll test our current system by asking a question that requires up-to-date information:

<div style="background: linear-gradient(135deg, #3f4c6b 0%, #606c88 100%); padding: 15px; border-radius: 10px; color: white; margin: 10px 0;">
<strong>Test Question:</strong> "When is the next Oasis concert?"
</div>

### Why This Question is Perfect for Testing:

| Aspect | Challenge | Why It Matters |
|--------|-----------|----------------|
| **Time-Sensitive** | Concert dates change frequently | Tests real-time information needs |
| **Current Events** | Requires recent announcements | Beyond training data cutoff |
| **Domain-Specific** | Music industry knowledge | Tests specialized information |
| **External Sources** | Needs web search capability | Tests external data access |

**Expected Behavior**: Our current system will likely provide generic or outdated information since it lacks access to current concert listings and tour announcements.

## 3.2 Current System Test

Let's see how our existing AgentQnA system handles this real-time information request:

In [ ]:
# Make the question
!curl -s http://localhost:9090/v1/chat/completions \
  -X POST \
  -H "Content-Type: application/json" \
  -d '{"role": "user", "messages": "When is the next Oasis concert?", "thread_id": "random_2", "stream": false}' \
  | jq -r '.text'

In [ ]:
# Let's check the supervisor logs

!kubectl logs -l app.kubernetes.io/name=supervisor --tail=30


## 3.3 Analysis of Current System Limitations

As we can see from the response above, our current system has several key limitations when dealing with real-time information:

### Detailed Analysis:

| Limitation | Root Cause | Business Impact |
|------------|------------|-----------------|
| **Outdated Knowledge** | Static training data with cutoff date | Inaccurate responses to current events |
| **No External Access** | Isolated system without internet connectivity | Cannot access real-time information |
| **Limited Tool Set** | No tools for dynamic data retrieval | Stuck with pre-existing knowledge only |
| **Static Response** | Cannot adapt to changing information | Potentially misleading or false information |

**Key Insight**: The system performs well with static, historical information but fails when current, dynamic data is required.

---



# 4. Solution Architecture: Web Search Agent



## 4.1 Architectural Overview

To solve these limitations, we'll create a comprehensive **Web Search Agent** that integrates seamlessly with our existing system:

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 15px; border-radius: 10px; color: white; margin: 10px 0;">
<strong>Our Solution Components:</strong>
<ul>
<li><strong>Google Search API Integration</strong> - Real-time web search capabilities</li>
<li><strong>Agent Tool Definition</strong> - YAML interface for supervisor routing</li>
<li><strong>Python Implementation</strong> - Search logic and result processing</li>
<li><strong>Microservice Deployment</strong> - Kubernetes-based agent service</li>
<li><strong>Supervisor Integration</strong> - Seamless routing and orchestration</li>
</ul>
</div>

### Implementation Roadmap:

1. **Tool Interface Design** - Define agent capabilities and parameters
2. **Search Implementation** - Build Google API integration
3. **Supervisor Integration** - Enable agent discovery and routing
4. **Microservice Deployment** - Add new service to Kubernetes cluster
5. **Testing & Validation** - Verify real-time capabilities

Let's break down each step in detail:

## 4.2 Understanding the Agent Tools Architecture

### Tools Directory Structure

Agent capabilities in OPEA are defined through a **dual-file architecture** that separates interface from implementation:

<div style="background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%); padding: 15px; border-radius: 10px; color: #333; margin: 10px 0;">
<strong>File Types & Purposes:</strong>
</div>

| File Type | Purpose | Contains | Example |
|-----------|---------|----------|---------|
| **YAML Files** (`.yaml`) | **Interface Definition** | Tool schemas, parameters, descriptions | `worker_web_agent_tools.yaml` |
| **Python Files** (`.py`) | **Implementation Logic** | Functions, API calls, business logic | `worker_web_agent_tools.py` |

### Detailed Component Analysis:

#### YAML Configuration Files:
- **Tool Names**: Unique identifiers for each capability
- **Descriptions**: Help supervisor understand when to use tools
- **Parameters**: Input/output schemas and validation
- **Function Mapping**: Links to Python implementation

#### Python Implementation Files:
- **Function Definitions**: Actual tool logic and algorithms
- **API Integrations**: External service connections (Google, etc.)
- **Data Processing**: Result formatting and filtering
- **Error Handling**: Robust failure management

**Design Philosophy**: This separation allows for easy maintenance, testing, and updates without breaking the agent interface.

### Directory Structure in OPEA Agents

The tools for both **Supervisor** and **Worker** agents are located in the `/mnt/tools/` directory. This directory is mounted in the Kubernetes cluster and accessible to all agents.

<div style="background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%); padding: 15px; border-radius: 10px; color: #333; margin: 10px 0;">
<strong>Standard Directory Layout:</strong>
</div>

```bash
/mnt/tools/
├── supervisor_agent_tools.yaml    # Supervisor: Tool interface definitions
├── supervisor_agent_tools.py      # Supervisor: Tool implementations
├── tools.py                       # Shared utilities and common functions + SQL and CRAG in the AgentQnA example
├── worker_agent_tools.yaml          # RAG Agent: Knowledge base interface
├── worker_agent_tools.py            # RAG Agent: Search implementations
```

### What We'll Add Today:

| File | Purpose | What It Does |
|------|---------|--------------|
| `worker_web_agent_tools.yaml` | **Web Search Interface** | Defines search capabilities and parameters |
| `worker_web_agent_tools.py` | **Search Implementation** | Implements Google API integration |
| Update `supervisor_agent_tools.yaml` | **Supervisor Enhancement** | Adds web search to available tools |
| Update `tools.py` | **Integration Logic** | Connects supervisor to web agent |


## 4.3 Creating the Web Search Agent

Now that we understand the directory structure, let's create our web search agent. We'll need to create two new files:

1. `worker_web_agent_tools.yaml`: Defines the interface for our web search functionality
2. `worker_web_agent_tools.py`: Implements the actual web search using Google's API

### Create the Interface Definition

First, let's create the YAML file that defines our web search tool. This file will:
- Define a `search_concert_schedule` function
- Specify the input parameters (search query)
- Define the expected output format
- Map to the implementation in our Python file

Navigate to `/mnt/tools/` and create a new file called `worker_web_agent_tools.yaml` with the following content:

```yaml
search_concert_schedule:
  description: Search in the internet for recent concert schedule information.
  callable_api: worker_web_agent_tools.py:search_web_base
  args_schema:
    query:
      type: str
      description: query
  return_output: retrieved_data
```

Click on `Save`

### Implement the Search Functionality

Next, let's create the Python implementation file `worker_web_agent_tools.py`. This file will:
- Connect to Google's Search API
- Process search results
- Return formatted concert information

Remain the same `/mnt/tools/`folder create `worker_web_agent_tools.py` with the following implementation:

```python
def search_web_base(query: str) -> str:
    import os
    api_key = os.environ["GOOGLE_API_KEY"]
    cse_id = os.environ["GOOGLE_CSE_ID"]

    from langchain_core.tools import Tool
    from langchain_google_community import GoogleSearchAPIWrapper

    search = GoogleSearchAPIWrapper()

    tool = Tool(
        name="google_search",
        description="Search Google for the MOST recent results.",
        func=search.run,
    )

    response = tool.run(query)
    return response
```

This implementation:
1. Uses environment variables for API credentials
2. Leverages LangChain's Google Search wrapper
3. Creates a search tool that returns formatted results
4. Handles the query and response processing


### Implement Supervisor Integration

Replace `search_web_base` function for the following function on `mnt/tools/` folder present on `tools.py` to handle web search requests:

```python
@opea_telemetry
def search_web_base(query: str) -> str:
    url = os.environ.get("WORKER_NAVIGATION_URL")
    proxies = {"http": ""}
    
    payload = {
        "messages": [
            {"role": "user", "content": query}
        ]
    }
    
    response = requests.post(url, json=payload, proxies=proxies)
    data = response.json()
    
    return data.get("text", "")
```

This implementation:
1. Uses the worker's agent URL from environment variables
2. Formats the query as a chat message
3. Handles the API communication with the worker
4. Processes and returns the response

#### Open `supervisor_agent_tools.yaml` and add on the top of the file the worker information to let the supervisor know.

```bash 
search_concert_schedule:
  description: Search in the internet for recent concert schedule information.
  callable_api: tools.py:search_web_base
  args_schema:
    query:
      type: str
      description: query
  return_output: retrieved_data
```

### Set the New Agent Microservice

Naviagate to the HELM chart configuration files `/agentqna` (`.../Kubernetes/AgentQnA/agentqna`), open `values.yaml` and look for the `supervisor` component. Replace the `supervisor` component and add the new browser agent configuration.


```yaml
browseragent:                                                   #<------ Component Name
  toolHostPath: "/mnt/tools"
  service:
    port: 9097                                                          #<------ PORT for our new component (BE SURE DONT OVERLAP, you could use 9097)
  strategy: react_llama                                                 #<------ Strategy for the component (refer to :https://github.com/opea-project/GenAIComps/blob/main/comps/agent/src/README.md#12-llm-engine))
  with_memory: true
  recursion_limit: 10
  llm_engine: openai
  temperature: "0"
  max_new_tokens: "4096"
  stream: "true"
  tools: /home/user/tools/worker_web_agent_tools.yaml                    #<------ Path were the agent will read the instructions locally(yaml)
  require_human_feedback: false
######VALIDATE
  GOOGLE_API_KEY: ""
  GOOGLE_CSE_ID: ""

supervisor:
  toolHostPath: "/mnt/tools"
  service:
    port: 9090
  strategy: react_llama
  with_memory: true
  recursion_limit: 10
  llm_engine: vllm
  llm_endpoint_url: ""
  model: "meta-llama/Llama-3.3-70B-Instruct"
  temperature: "0"
  max_new_tokens: "4096"
  stream: "true"
  tools: /home/user/tools/supervisor_agent_tools.yaml
  require_human_feedback: false
  CRAG_SERVER: ""
  WORKER_AGENT_URL: ""
  SQL_AGENT_URL: ""
###VALIDATE
  WORKER_NAVIGATION_URL: ""                                             # <-------------ADD THIS LINE AT THE ENDSpecify where the Browser worker is located

```

#### In the same folder open `variant_openai-values.yaml` and configure your LLMs platform (You can mix them to run on multiple platforms!)


```yaml
supervisor:
  model: "meta-llama/Llama-3.3-70B-Instruct
  llm_endpoint_url: "https://api.inference.denvrdata.com"
  OPENAI_API_KEY: "your_denvr_key"

ragagent:
  model: "meta-llama/Llama-3.3-70B-Instruct"
  llm_endpoint_url: "https://api.inference.denvrdata.com"
  OPENAI_API_KEY: "your_denvr_key"

sqlagent:
  model: "meta-llama/Llama-3.3-70B-Instruct"
  llm_endpoint_url: "https://api.inference.denvrdata.com"
  OPENAI_API_KEY: "your_denvr_key"

browseragent:
  #model: "gpt-4o-mini-2024-07-18"
  model: "meta-llama/Llama-3.3-70B-Instruct"
  llm_endpoint_url: "https://api.inference.denvrdata.com"
  llm_engine: openai
  OPENAI_API_KEY: "your_denvr_key"
```

####  Go to `.../Kubernetes/AgentQnA/agentqna/Chart.yaml` and add the new microservice config `worker-browser-agent` right after `dependencies:`
```bash
- alias: browseragent
  name: agent
  repository: file://../common/agent
  version: 0-latest
```

Your new config should look like : 

![image](./Images/agent_2.png)

#### Modify the config map to use declare the environment variables to be passed. 

In [ ]:
file_path = "agentqna/charts/agent/templates/configmap.yaml"

helm_block = '''\
{{- if .Values.WORKER_NAVIGATION_URL }}
WORKER_NAVIGATION_URL: {{ .Values.WORKER_NAVIGATION_URL | quote }}
{{- else }}
WORKER_NAVIGATION_URL: "http://{{ .Release.Name }}-browseragent:9097/v1/chat/completions"
{{- end }}

{{- if .Values.GOOGLE_API_KEY }}
GOOGLE_API_KEY: {{ .Values.GOOGLE_API_KEY | quote }}
{{- else }}
GOOGLE_API_KEY: "default-or-empty-value"
{{- end }}

{{- if .Values.GOOGLE_CSE_ID }}
GOOGLE_CSE_ID: {{ .Values.GOOGLE_CSE_ID | quote }}
{{- else }}
GOOGLE_CSE_ID: "default-or-empty-value"
{{- end }}  
'''

with open(file_path, "r") as f:
    lines = f.readlines()

new_lines = []
inserted = False
for line in lines:
    new_lines.append(line)
    if not inserted and line.strip() == "data:":
        indent = "  "  # 2 spaces indent to match YAML
        for helm_line in helm_block.split("\n"):
            new_lines.append(indent + helm_line + "\n")
        inserted = True

with open(file_path, "w") as f:
    f.writelines(new_lines)

print("✅ Helm block inserted only after 'data:' with correct indentation")

# 5. Google Search API Configuration



## 5.1 Why Google Search API?

Before we implement our web search agent, we need access to Google's powerful search capabilities. Google's **Custom Search API** provides:

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 15px; border-radius: 10px; color: white; margin: 10px 0;">
<strong>Key Benefits:</strong>
<ul>
<li><strong>Real-Time Results</strong> - Access to current web information</li>
<li><strong>Fast Performance</strong> - Optimized search infrastructure</li>
<li><strong>Customizable</strong> - Filter results by site, date, content type</li>
<li><strong>Structured Data</strong> - JSON responses perfect for agents</li>
<li><strong>Reliable</strong> - Enterprise-grade API with rate limiting</li>
</ul>
</div>

### Setup Requirements:

| Component | Purpose | What You Get |
|-----------|---------|--------------|
| **Google Cloud Project** | API management and billing | Organized resource container |
| **API Key** | Authentication credentials | Access to Google services |
| **Custom Search Engine** | Search scope configuration | Tailored search results |

---

## 5.2 Step-by-Step API Setup 

To enable our web search agent, we need two essential credentials from Google:

<div style="background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%); padding: 15px; border-radius: 10px; color: #333; margin: 10px 0;">
<strong>Required Credentials:</strong>
<ul>
<li><strong>GOOGLE_API_KEY</strong> - Authenticates API requests to Google services</li>
<li><strong>GOOGLE_CSE_ID</strong> - Identifies your custom search engine configuration</li>
</ul>
</div>

**Security Note**: These credentials will be securely stored in your Kubernetes cluster as environment variables and never exposed in your code.

### 5.2.1 Create Google Cloud Project

Go to the [Google Cloud Console](https://console.cloud.google.com/welcome/new?inv=1&invt=Abyc4w).

- Click *Create or Select a project* button.

- On the top bar and then *New Project*.

Name your project (e.g., “opea-agents”) and click **Create**

![image](./Images/gc_1.png)

Click on your navigation menu

![Image](./Images/navigation.png)

Inside your project dashboard, go to the Navigation Menu (☰) > APIs & Services > Credentials.

![Image](./Images/apis.png)

Look for the  “+ Create credentials” button.

Click on `API Key`

![Image](./Images/api_key.png)

Copy your key for later. This key will be your `GOOGLE_API_KEY`

![IMages](./Images/save_key.png)

### 4.3 Set Up a Custom Search Engine (CSE) to Get Your GOOGLE_CSE_ID

Go to [Google Custom Search Engine](https://programmablesearchengine.google.com/controlpanel/all).

- Click `Add` to create a new search engine.

In Sites to search, you can start by typing www.google.com or leave it empty to search the entire web (but it may require some tweaks).

- Name your search engine (e.g., “opea_demo”).

Click `Create`.

![image](./Images/search_eng2.png)

After creating, Click on *Customize*.

![image](./Images/custom_2.png)

Find the *Search engine ID* — this is your `GOOGLE_CSE_ID`.

![image](./Images/cse_id.png)


(Optional) In the Sites to search section, to enable searching the entire web, set Search the entire web but emphasize included sites by toggling the option.

# 6. Deploy and test

In [ ]:
import os

# Define local tools directory
WORKDIR = os.getcwd()
TOOLS_DIR = os.path.join(WORKDIR, "mnt/tools")

if os.path.exists(TOOLS_DIR) and os.path.isdir(TOOLS_DIR):
    # Create destination folder in the kind node
    os.system("docker exec kind-control-plane mkdir -p /mnt/tools")

    # Copy everything inside mnt/tools/ into kind-control-plane:/mnt/tools/
    os.system(f"docker cp {TOOLS_DIR}/. kind-control-plane:/mnt/tools/")

    print("✅ All tools copied into kind-control-plane:/mnt/tools/")
else:
    print(f"❌ Directory {TOOLS_DIR} does not exist or is not a directory. Skipping copy.")


In [ ]:
# Deploy helm chart using OpenAI
!helm upgrade --install agentqna agentqna \
  -f agentqna/variant_openai-values.yaml \
  --set browseragent.GOOGLE_API_KEY="GOOGLE_API_KEY" \
  --set browseragent.GOOGLE_CSE_ID="GOOGLE_CSE_ID"\

In [ ]:
# Restart supervisor pod to get the changes

# Restart the Supervisor agent to load the updated configuration
# This ensures the agent picks up our new tool descriptions

print("🔄 Restarting Supervisor agent to apply changes...")
!kubectl rollout restart deployment/agentqna-supervisor

print("⏳ Waiting for rollout to complete...")
!kubectl rollout status deployment/agentqna-supervisor --timeout=60s

print("✅ Supervisor agent restarted with updated configuration!")

In [ ]:
# First, let's ensure our Supervisor agent is accessible
# This assumes you have completed Notebook 1 and have AgentQnA running

import subprocess
import socket
import time

def is_port_open(host='localhost', port=9090):
    """Check if port is already open"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.settimeout(1)
        try:
            sock.connect((host, port))
            return True
        except:
            return False

if is_port_open():
    print("✅ Port-forward already running on localhost:9090")
else:
    print("🔁 Starting port-forward to Supervisor agent...")
    subprocess.Popen(
        ["kubectl", "port-forward", "svc/agentqna-supervisor", "9090:9090"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    time.sleep(2)
    if is_port_open():
        print("✅ Supervisor agent now accessible on localhost:9090")
    else:
        print("❌ Failed to start port-forward")

In [ ]:
!curl -s http://localhost:9090/v1/chat/completions \
  -X POST \
  -H "Content-Type: application/json" \
  -d '{"role": "user", "messages": "When is the next Oasis concert?", "thread_id": "random_2213", "stream": false}' \
  | jq -r '.text'

In [ ]:
# Lets check the supervisor logs 

!kubectl logs -l app.kubernetes.io/name=supervisor --tail=30

In [ ]:
# Lets check the browser agent logs 

!kubectl logs -l app.kubernetes.io/name=browseragent --tail=30

# 🎉 Congratulations!

You have successfully extended your OPEA AgentQnA system with real-time web search capabilities! You now have:

✅ **A powerful web search agent** with Google API integration  
✅ **Real-time information access** for current events and data  
✅ **Seamless supervisor integration** with intelligent routing  
✅ **Production-ready deployment** in your Kubernetes cluster  
✅ **Extensible architecture** ready for additional agents

### 🔄 What You've Accomplished

This notebook demonstrated the complete agent extension workflow:

1. **🧪 Problem Identification** - Analyzed limitations of static knowledge systems
2. **🏗️ Architecture Design** - Planned a comprehensive web search solution
3. **⚙️ Tool Implementation** - Built YAML interfaces and Python logic
4. **🔌 System Integration** - Connected new agent with supervisor orchestration
5. **🚀 Microservice Deployment** - Added new services to Kubernetes cluster
6. **🧪 Validation Testing** - Verified real-time information capabilities

### 🚀 Next Steps & Extensions

Now that you understand the agent extension process, you can:

- **📰 News Agent**: Create agents that fetch breaking news and current events
- **💰 Financial Agent**: Build agents with real-time market data and stock prices
- **🌤️ Weather Agent**: Develop agents with current weather and forecast information
- **📊 Analytics Agent**: Create agents that pull real-time metrics and KPIs
- **🔗 Multi-API Agents**: Combine multiple external APIs in single agents

### 💡 Key Architectural Insights

- **🧩 Modular Design**: Each agent is independent and can be developed separately
- **🔄 Hot Deployment**: New agents can be added without system downtime  
- **📈 Scalable Architecture**: The supervisor pattern scales to many specialized agents
- **🛡️ Security First**: API credentials are managed securely through Kubernetes secrets
- **🧠 Intelligent Routing**: The supervisor automatically routes queries to appropriate agents

### 🛠️ Production Considerations

For production deployments, consider:

- **🔒 Security**: Use Kubernetes secrets for API credentials
- **📊 Monitoring**: Add logging and metrics for agent performance
- **⚡ Caching**: Implement response caching for frequently requested data
- **🔄 Rate Limiting**: Manage API quotas and request limits
- **🧪 Testing**: Create comprehensive test suites for agent reliability

---

**Ready to build your next intelligent agent system?** The foundation is now in place!
